# MOF Screening using Active Learning #

In [1]:
import pandas as pd
import os
import torch
from sklearn.preprocessing import StandardScaler
import numpy as np
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition.analytic import ExpectedImprovement

In [2]:
# Get the current working directory and read the csv file into the notebook
PATH = os.getcwd()
CSV_FILE = os.path.join(PATH, "./Combined_set_prescreened (1).csv");
df = pd.read_csv(CSV_FILE);
df.head()

,MOF,uptake_ads,uptake_des,heat_ads,heat_des,LISD,LFSD,LISFS,Unit_cell_volume,Density,...,O,La,Cr,Ti,Ba,Rh,Ce,Cu,Al,Re
0,XUKYEI_neutral,0.920997,0.668303,-21.55231,-18.39099,13.18217,10.20370,13.18217,6140.00,0.287208,...,0,0,0,0,0,0,0,2,0,0
1,ja300034j_si_002_clean,0.433112,0.187346,-25.96441,-17.79542,17.49700,17.44104,17.49700,2800.68,0.713223,...,18,0,0,0,0,0,0,0,0,0
2,QIYDAF01_clean,0.827077,0.660425,-19.37443,-18.95302,22.00141,13.48659,22.00141,52812.60,0.303251,...,96,0,0,0,0,0,0,24,0,0
3,XAHPIH_clean,0.818678,0.634451,-21.96228,-19.54395,14.37026,13.22660,14.37026,12821.80,0.356183,...,40,0,0,0,0,0,0,8,0,0
4,VETMIS_clean,0.932519,0.746597,-22.10557,-19.92746,18.13430,11.96931,18.13430,33152.20,0.311959,...,48,0,0,0,0,0,0,12,0,0


### COP CALCULATION ###

In [3]:
df["deltaQ"] = df["uptake_ads"] - df["uptake_des"]
df["deltaHmean"] = (df["heat_ads"] + df["heat_des"])/2
df = df[df["deltaQ"]>=0]
deltaHvap = 16.25 
Tads = 313.15 
Tdes = 358.15
Mw = 44.097/1000
Cp = 1
df["COP"] = (deltaHvap * df["deltaQ"]) / (Mw*Cp*(Tdes - Tads) - df["deltaHmean"]*df["deltaQ"]);
df.drop(columns=["MOF", "uptake_ads", "uptake_des", "heat_ads", "heat_des", "deltaQ", "deltaHmean"], inplace=True) 

### ACTIVE LEARNING CODE ###

In [4]:
# Drop the rows with missing values
df.dropna(inplace=True)
# Drop the column that is repeated
df.drop(columns=["Number_of_pockets.1"], inplace=True)
# Drop the column that has only one unique value
df.drop(columns=["Pu"], inplace=True)

In [5]:
# Display all the 107 columns in the dataframe
pd.set_option('display.max_columns', 108)

In [6]:
# Find the name of all the columns in the dataframe as a list
features = df.columns.tolist()
features

['LISD',
 'LFSD',
 'LISFS',
 'Unit_cell_volume',
 'Density',
 'ASA_A2',
 'ASA_m2_per_cm3',
 'ASA_m2_per_g',
 'NASA_A2',
 'NASA_m2_per_cm3',
 'NASA_m2_per_g',
 'Number_of_channels',
 'Channel_surface_area_A2',
 'Number_of_pockets',
 'AV_A3',
 'AV_Volume_fraction',
 'AV_cm3_per_g',
 'NAV_A3',
 'NAV_Volume_fraction',
 'NAV_cm3_per_g',
 'Channel_volume',
 'oms_value',
 'Pocket_volume_A3',
 'Pocket_surface_area_A2',
 'No_of_Channel',
 'Di',
 'Df',
 'Dif',
 'H',
 'Yb',
 'Ir',
 'K',
 'V',
 'Te',
 'Mn',
 'Co',
 'Sr',
 'Bi',
 'Cd',
 'Mg',
 'P',
 'Be',
 'Cl',
 'As',
 'F',
 'Pb',
 'Pr',
 'Mo',
 'I',
 'Er',
 'Fe',
 'Y',
 'W',
 'Gd',
 'Sn',
 'Se',
 'Cs',
 'B',
 'Ge',
 'C',
 'Si',
 'Ag',
 'Pt',
 'S',
 'Th',
 'Ca',
 'Dy',
 'Au',
 'Rb',
 'Zr',
 'Ho',
 'Br',
 'Sc',
 'Na',
 'U',
 'Pd',
 'Nd',
 'Ni',
 'Np',
 'N',
 'Nb',
 'Ru',
 'Li',
 'Lu',
 'Zn',
 'Hg',
 'Hf',
 'Sb',
 'In',
 'Eu',
 'Ga',
 'Tb',
 'Tm',
 'Sm',
 'O',
 'La',
 'Cr',
 'Ti',
 'Ba',
 'Rh',
 'Ce',
 'Cu',
 'Al',
 'Re',
 'COP']

In [7]:
# Remove the COP column from the list of columns
X = df[features[:-1]].values
print("Shape of the feature matrix: ", X.shape)
y = df["COP"].values
print("Shape of the target matrix: ", y.shape)

Shape of the feature matrix:  (6917, 104)
Shape of the target matrix:  (6917,)


In [8]:
# Standardize the data in X
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [9]:
# Convert the data from the numpy array to a tensor
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [10]:
# Check the shape of the data
X.size(), y.size()

(torch.Size([6917, 104]), torch.Size([6917]))

In [11]:
# The forward function of Expected Improvement requires the input tensor [(b1xb2xb3....xbk)x1xd dim] and return tensor d dim
X_unsqueezed = X.unsqueeze(1)

### Bayesian Optimization Function ###

In [13]:
# Number of MOFs in the dataset
numberOfMOFs = 6917
# Creating the Bayesian Optimazaition function
def bayesianOptimizationFunction(numberOfIterations, numberOfSamples):
    # Ensuring the number of iterations is greater than the number of samples so that model can be trained extensively
    assert numberOfIterations > numberOfSamples
    # Initial data points for the Bayesian Optimization
    initialDataPoints = np.random.choice(numberOfMOFs, numberOfSamples, replace=False)
    # Initial X and y values
    initialY = y[initialDataPoints]
    initialX = X[initialDataPoints]
    for i in range(numberOfSamples, numberOfIterations):
        # Create the model and the likelihood
        model = SingleTaskGP(initialX, initialY)
        modelLikelihood = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_mll(modelLikelihood)
        # Setting up the acquisition function
        # `best_f`= Best function value observed
        EI = ExpectedImprovement(model, best_f=initialY.max().item())
        with torch.no_grad():
            # Calculate the expected improvement uncertainity for dataset
            newValues = EI.forward(X_unsqueezed)
        # Arranging the indices in the descending order of expected improvement
        newDataPoints = newValues.argsort(descending=True)
        # Selecting datapoints which are initially not present
        for dataPoint in newDataPoints:
            if not dataPoint.item() in initialDataPoints:
                newMaxDataPoint = dataPoint.item()
                break
        initialDataPoints = np.concatenate([initialDataPoints, [newMaxDataPoint]])
        assert initialDataPoints.size == i+1
        # Updating new datapoints to y and X
        initialX = X[initialDataPoints]
        initialY = y[initialDataPoints]
    assert np.size(initialDataPoints) == numberOfIterations
    return initialDataPoints
        

In [15]:
index = bayesianOptimizationFunction(15, 10)

BotorchTensorDimensionError: An explicit output dimension is required for targets. Expected Y with dimension 2 (got Y.dim()=1).